In [46]:
import pandas as pd
import csv
import numpy as np
from itertools import product
from pulp import *
import re

In [137]:
player_list.loc[player_list['ID']==19640069]['Name'][0]

'Tom Brady'

In [160]:
player_list = pd.read_csv('C:/Users/Rob/Downloads/DKSalaries (3).csv')

player_list = player_list[(player_list['Roster Position']!= 'CPT')&(player_list['Name']!= 'Kareem Hunt')&(player_list['Name']!= 'Dallas Goedert')]
player_list['Position'] = player_list['Roster Position']
player_list = player_list.groupby(['ID','Name','AvgPointsPerGame','Salary','Position']).count().reset_index()


In [161]:
player_list

,ID,Name,AvgPointsPerGame,Salary,Position,Name + ID,Roster Position,Game Info,TeamAbbrev
0,19727218,Baker Mayfield,17.09,11400,FLEX,1,1,1,1
1,19727219,Teddy Bridgewater,18.03,11000,FLEX,1,1,1,1
2,19727220,Nick Chubb,17.96,10400,FLEX,1,1,1,1
3,19727221,Courtland Sutton,16.35,10000,FLEX,1,1,1,1
4,19727222,Case Keenum,0.12,9400,FLEX,1,1,1,1
5,19727224,Noah Fant,12.55,8200,FLEX,1,1,1,1
6,19727225,Jerry Jeudy,13.20,8000,FLEX,1,1,1,1
7,19727226,Tim Patrick,12.90,7600,FLEX,1,1,1,1
8,19727227,Melvin Gordon III,12.18,7400,FLEX,1,1,1,1
9,19727228,Javonte Williams,9.67,7000,FLEX,1,1,1,1


In [162]:
salaries = {}
points = {}

for pos in player_list['Position'].unique():
    available_pos = player_list[player_list['Position'] == pos]
    salary = list(available_pos[['ID','Salary']].set_index('ID').to_dict().values())[0]
    point = list(available_pos[['ID','AvgPointsPerGame']].set_index('ID').to_dict().values())[0]
    
    salaries[pos] = salary
    points[pos] = point

pos_num_available = {'FLEX':6}
# pos_num_available = {'QB':1, 'RB': 3, 'WR': 3, 'TE': 1, 'DST':1}
salary_cap = 44000

In [163]:
final_lineups = []
for lineup in range(1,101):
    _vars = {k: LpVariable.dict(k,v,cat='Binary') for k, v in points.items()}
    
    prob = LpProblem('Fantasy',LpMaximize)
    
    rewards = []
    costs = []
    position_constraints = []
    
    for k,v in _vars.items():
        costs += lpSum([salaries[k][i] * _vars[k][i] for i in v])
        rewards += lpSum([points[k][i] * _vars[k][i] for i in v])
        prob += lpSum([_vars[k][i] for i in v]) == pos_num_available[k]
        
    prob += lpSum(rewards)
    prob += lpSum(costs) <= salary_cap
    
    if not lineup == 1:
        prob += (lpSum(rewards) <= total_score-0.001)
        
    prob.solve()
    
    score = str(prob.objective)
    
    constaints = [str(const) for const in prob.constraints.values()]
    
    
    new_lineup = []
    new_dict = {}
    for v in prob.variables():
        
        score = score.replace(v.name, str(v.varValue))
        if v.varValue != 0 :
            new_lineup.append(v.name)
    
    total_score = eval(score)
    
    new_dict['players'] = new_lineup
    new_dict['score'] = total_score
    
    final_lineups.append(new_dict)
    


In [164]:
for i in final_lineups:
    n_lineup = []
    for n in i['players']:
        id_num = n.replace('TE_','').replace('WR_','').replace('DST_','').replace('QB_','').replace('RB_','').replace('FLEX_','')
        n_lineup.append(list(player_list[player_list['ID']== int(id_num)]['Name'])[0])
    print(n_lineup)


['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Jerry Jeudy', 'Brandon McManus', 'Albert Okwuegbunam']
['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Tim Patrick', 'Brandon McManus', 'Albert Okwuegbunam']
['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Jerry Jeudy', 'Chase McLaughlin', 'Albert Okwuegbunam']
['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Tim Patrick', 'Chase McLaughlin', 'Albert Okwuegbunam']
['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Noah Fant', 'Brandon McManus', 'Albert Okwuegbunam']
['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Jerry Jeudy', 'Brandon McManus', 'KJ Hamler']
['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Noah Fant', 'Chase McLaughlin', 'Albert Okwuegbunam']
['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Melvin Gordon III', 'Brandon McManus', 'Albert Okwuegbunam']
['Baker Mayfield', 'Teddy Bridgewater', 'Nick Chubb', 'Tim Patrick', 'Broncos ', 'Albert Okwuegbunam']
['Ted

['Teddy Bridgewater', 'Nick Chubb', 'Courtland Sutton', 'Jerry Jeudy', 'Browns ', 'Albert Okwuegbunam']
